In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings

from data_describe.text.text_preprocessing import *
from sklearn.datasets import fetch_20newsgroups

warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append("../scripts/")
from data_cleaner import DataCleaner
from util import Util

In [3]:
util = Util()
cleaner = DataCleaner()

In [30]:
repo="https://github.com/Nathnael12/prompt-engineering.git"
cleaned_df=util.read_from_dvc("data/news.csv",repo,"news-v1",low_memory=False)

# cleaned_df=util.read_from_file('../data/news.csv')

In [5]:
cleaner.get_numerical_columns(cleaned_df)

['Analyst_Rank', 'Reference_Final_Score', 'Analyst_Average_Score']

In [18]:
def tokenizer(df:pd.DataFrame,columns:list):
    df2=df.copy()
    for col in columns:
        df2[col]=df2[col].apply(lambda x: to_list(tokenize(x)))
    return df2

In [19]:
categorical_cols=cleaner.get_categorical_columns(cleaned_df)
tokens=tokenizer(cleaned_df,categorical_cols)

In [28]:
to_list(to_lower("addis neger"))

[['a'], ['d'], ['d'], ['i'], ['s'], [' '], ['n'], ['e'], ['g'], ['e'], ['r']]